In [1]:
from kafka import KafkaConsumer
import pandas as pd
from influxdb_client import InfluxDBClient, Point, WritePrecision
from json import loads
from influxdb_client.client.write_api import SYNCHRONOUS
import numpy as np
from sklearn.preprocessing import StandardScaler
import pickle
from datetime import datetime
import joblib

In [2]:


# Kafka broker address
bootstrap_servers = 'localhost:9093'
# Kafka topic to consume messages from
kafka_topic = 'QM_data_class'
print("Connecting to consumer ...")

# Consumer group ID
group_id = 'voting-consumer'
# InfluxDB connection details
influxdb_host = 'localhost'
influxdb_port = 8086
influxdb_username = 'Rahma'
influxdb_password = 'rahma123'
bucket = "BloodPressure"
organization = "OstProject"
influxdb_token = "NwFT1hvPZogYtaZG4OCUjF0_BJ46riO_DFFXPW6VaHkWQBFcNQdh67nfT6pPuYUH2kWhFDtiY1Cl2BSRAA4VyQ=="


model_path = './v_model.pkl'

with open(model_path, 'rb') as model_file:
    ensemble_model = joblib.load(model_file)
influx_client = InfluxDBClient(url="http://localhost:8086", token=influxdb_token, org=organization,bucket=bucket
                               )
true_labels = []
predicted_labels = []

# Load true labels from CSV
test_df= pd.read_csv("./real_time_data.csv")

true_labels_df = test_df['Class']

Connecting to consumer ...


In [4]:

from sklearn.metrics import confusion_matrix
with InfluxDBClient(url="http://localhost:8086", token=influxdb_token, org=organization,bucket=bucket) as client:
        write_api = client.write_api(write_options=SYNCHRONOUS)
        consumer = KafkaConsumer(kafka_topic, bootstrap_servers=['localhost:9093'],auto_offset_reset='earliest', api_version=(0,10),enable_auto_commit=True,value_deserializer=lambda x: loads(x.decode('utf-8')))
        print("Connection established")
        i=0
        m=0
        correctly_classified=0
        wrongfully_classified=0

        for message in consumer:
            message_data = message.value
            
            timestamp_str = message_data['Timestamp'] 
            true_label = true_labels_df.loc[m]
            


            message_features = [float(message_data['RR']), float(message_data['SPO2']),
                    float(message_data['MAP']), float(message_data['SBP']),
                    float(message_data['DBP']), float(message_data['HR']),
                    float(message_data['PP']), float(message_data['CO'])]

# Specify the correct feature names
            feature_names = ['RR', 'SPO2', 'MAP', 'SBP', 'DBP', 'HR', 'PP', 'CO']

# Create a DataFrame with correct column names
            features_df = pd.DataFrame([message_features], columns=feature_names)
            # Predict using the ensemble model
            predicted_class = ensemble_model.predict(features_df)
            # Predict using the ensemble model
            predicted_class = int(predicted_class[0]) 
            print (predicted_class)

            if predicted_class == true_label:
                  correctly_classified +=1
            else:
                  wrongfully_classified +=1

                  
            #cm = confusion_matrix(true_label, predicted_class) 
            # tn, fp, fn, tp = cm.ravel()
            # print (tn,fp,fn,tp)

                # Create a Point object
            point = Point("classModel")
            point.time(datetime.utcnow(), WritePrecision.NS)

            # Add fields to the point
            point.field("RR", float(message_data['RR']))
            point.field("SPO2", float(message_data['SPO2']))
            point.field("MAP", float(message_data['MAP']))
            point.field("SBP", float(message_data['SBP']))
            point.field("DBP", float(message_data['DBP']))
            point.field("HR", float(message_data['HR']))
            point.field("PP", float(message_data['PP']))
            point.field("CO", float(message_data['CO']))
            point.field("Predicted_Class", predicted_class)
            point.field("True_Class", true_label)
            point.field("Correct Classifications", correctly_classified)
            point.field("Wrong Classifications", wrongfully_classified)

            # Write the point to InfluxDB
            write_api.write(bucket, organization, point)

            i += 1
            m +=1
            print(f"Message sent to InfluxDB: {i}")
            if m >= 3864:
                   print("Data ingestion stopped. Maximum limit reached.")
                   break


Connection established
0
Message sent to InfluxDB: 1
0
Message sent to InfluxDB: 2
0
Message sent to InfluxDB: 3
0
Message sent to InfluxDB: 4
0
Message sent to InfluxDB: 5
0
Message sent to InfluxDB: 6
0
Message sent to InfluxDB: 7
0
Message sent to InfluxDB: 8
0
Message sent to InfluxDB: 9
0
Message sent to InfluxDB: 10
0
Message sent to InfluxDB: 11
0
Message sent to InfluxDB: 12
0
Message sent to InfluxDB: 13
0
Message sent to InfluxDB: 14
0
Message sent to InfluxDB: 15
0
Message sent to InfluxDB: 16
0
Message sent to InfluxDB: 17
0
Message sent to InfluxDB: 18
0
Message sent to InfluxDB: 19
0
Message sent to InfluxDB: 20
0
Message sent to InfluxDB: 21
0
Message sent to InfluxDB: 22
0
Message sent to InfluxDB: 23
0
Message sent to InfluxDB: 24
0
Message sent to InfluxDB: 25
0
Message sent to InfluxDB: 26
0
Message sent to InfluxDB: 27
0
Message sent to InfluxDB: 28
0
Message sent to InfluxDB: 29
0
Message sent to InfluxDB: 30
0
Message sent to InfluxDB: 31
0
Message sent to InfluxD